[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/trainer/blob/main/scenedetect.ipynb)

In [ ]:
!pip install -q yt_dlp scenedetect[opencv]

In [ ]:
from yt_dlp import YoutubeDL

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

In [ ]:
test = 'https://playlist.m3u8'
!ffmpeg -protocol_whitelist file,http,https,tcp,tls -allowed_extensions ALL -i {test} -c copy playlist.mp4 > /dev/null 2>&1

In [ ]:
video_id = 'TnGl01FkMMo'
download_video(video_id)
video_path = f"/content/videos/{video_id}.mp4"
# !scenedetect -i {video_path} -o /content/adaptive --drop-short-scenes detect-adaptive save-images
!scenedetect -i {video_path} -o /content/content --drop-short-scenes --start 00:01:20 --end 01:23:00 detect-content save-images
# !scenedetect -i {video_path} -o /content/threshold --drop-short-scenes detect-threshold save-images

In [ ]:
%cd /content/content
!zip -r content.zip ./

In [ ]:
%cd /content/content
!rm -rf /content/content/*.jpg
!unzip /content/content/content.zip -d ./

In [ ]:
import os, cv2
import numpy as np

folder_path = "/content/content"
file_list = os.listdir(folder_path)
for file_name in file_list:
    if file_name.lower().endswith((".jpg", ".jpeg", ".png", ".gif")):
        full_path = os.path.join(folder_path, file_name)
        try:
          image = cv2.imread(full_path)
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
          contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          largest_contour = max(contours, key=cv2.contourArea)
          x, y, w, h = cv2.boundingRect(largest_contour)
          cropped_image = image[y:y + h, x:x + w]
          cv2.imwrite(full_path, cropped_image)

          image = cv2.imread(full_path)
          gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          average_intensity = cv2.mean(gray_image)[0]
          if average_intensity < 20:
            print(average_intensity, full_path)
            os.remove(full_path)
        except Exception as e:
          os.remove(full_path)
          print(f"An error occurred while processing {full_path}: {str(e)}")

In [ ]:
%cd /content/content
!zip -r content_cropped.zip ./*.jpg